In [1]:
import gc
import sys
import os
import warnings
from tqdm import tqdm
import seaborn as sns    

#warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

tqdm.pandas()
import sys

sys.path.append(os.path.abspath(".."))

import modules
import importlib
importlib.reload(modules)

from modules.memory_utils import pandas_reduce_mem_usage, pandas_string_to_cat

In [2]:
LOCAL_DATA_PATH = '../data/'
SPLIT_SEED = 42

In [3]:
from fe_modules.datetime_features import get_timestamp
from geopy.distance import geodesic as GD
from fe_modules.geo_features import map_cities

In [4]:
df = pandas_reduce_mem_usage( 
        pd.read_parquet(
            f'{LOCAL_DATA_PATH}competition_data_final_pqt/part-00000-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet'
    ))
df = map_cities(df, folder_path= "../external_data/")
df = get_timestamp(df, date_col='date')
df = pandas_reduce_mem_usage(df)
#df['lat_lon'] = df[['geo_lat','geo_lon','city_name']].progress_apply(lambda r: tuple(r), axis=1).apply(np.array)
#df = df.sort_values('timestamp')

Memory usage of dataframe is 2988.16 MB


  0%|          | 0/12 [00:00<?, ?it/s]

Memory usage after optimization is: 2521.26 MB
Decreased by 15.6%
Memory usage of dataframe is 0.05 MB


  0%|          | 0/6 [00:00<?, ?it/s]

Memory usage after optimization is: 0.03 MB
Decreased by 47.8%


C:\Users\fedor\PycharmProjects\mts-ml-cup\fe_modules\datetime_features.py:25: FutureWarning: The behavior of .astype from datetime64[ns] to int32 is deprecated. In a future version, this astype will return exactly the specified dtype instead of int64, and will raise if that conversion overflows.
  df[alias] = pd.DatetimeIndex(df[date_col]).astype(int) / scaler


Memory usage of dataframe is 3614.32 MB


  0%|          | 0/18 [00:00<?, ?it/s]

Memory usage after optimization is: 3487.51 MB
Decreased by 3.5%


In [5]:
df.head()

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id,capital_marker,timezone,geo_lat,geo_lon,population,timestamp
0,Краснодарский край,Краснодар,Apple,iPhone 7,ad.adriver.ru,smartphone,iOS,20368.0,2022-06-15,morning,1,45098,2.0,3,45.040161,38.975964,744933,165525120.0
1,Краснодарский край,Краснодар,Apple,iPhone 7,apple.com,smartphone,iOS,20368.0,2022-06-19,morning,1,45098,2.0,3,45.040161,38.975964,744933,165559680.0
2,Краснодарский край,Краснодар,Apple,iPhone 7,avatars.mds.yandex.net,smartphone,iOS,20368.0,2022-06-12,day,1,45098,2.0,3,45.040161,38.975964,744933,165499200.0
3,Краснодарский край,Краснодар,Apple,iPhone 7,googleads.g.doubleclick.net,smartphone,iOS,20368.0,2022-05-16,day,1,45098,2.0,3,45.040161,38.975964,744933,165265920.0
4,Краснодарский край,Краснодар,Apple,iPhone 7,googleads.g.doubleclick.net,smartphone,iOS,20368.0,2022-05-30,day,1,45098,2.0,3,45.040161,38.975964,744933,165386880.0


In [7]:
def haversine_np(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

def get_dist(points):
    points =  np.stack(points.to_numpy())
    point_shifted = np.roll(points, 1,axis=0)
    point_shifted[0]=np.array([np.nan,np.nan,np.nan])
    temp = (point_shifted[:,2]!=points[:,2])
    points = np.delete(points[temp], 2, 1).astype(np.float32)[1:]
    point_shifted = np.delete(point_shifted[temp], 2, 1).astype(np.float32)[1:]
    return haversine_np(points[:,0],
                        points[:,1],
                        point_shifted[:,0],
                        point_shifted[:,1]).mean()

def get_agg_amount_of_travel(df: pd.DataFrame,
                             agg_col: str = "user_id",
                             target_col_lat: str = 'geo_lat',
                             target_col_lon: str = 'geo_lon',
                             timestamp_col: str = 'timestamp',
                             city_name_col: str='city_name',
                             alias: str = None,
                             sort: bool = False) -> pd.DataFrame:
    if alias:
        col_name = alias
    else:
        col_name = f'{agg_col}_mean_travel_distance'
    df['lat_lon'] = df[[target_col_lat,target_col_lon,city_name_col]].progress_apply(lambda r: tuple(r), axis=1).apply(np.array)
    
    df = df.merge(df.sort_values(timestamp_col).groupby(agg_col)['lat_lon']
                  .agg(geometry = get_dist).fillna(0).astype(np.float32)
                  .rename(columns={'geometry': col_name}), how='left', on=agg_col)
    
    del df['lat_lon']
    
    if sort:
        return df.sort_values(by=agg_col)

    return df

In [8]:
%%time
df = get_agg_amount_of_travel(df)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33244681/33244681 [02:23<00:00, 232307.85it/s]
C:\Users\fedor\AppData\Local\Temp\ipykernel_6352\822610942.py:23: RuntimeWarning: Mean of empty slice.
  point_shifted[:,1]).mean()
C:\Users\fedor\PycharmProjects\mts-ml-cup\venv\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


CPU times: total: 4min 28s
Wall time: 4min 29s


In [10]:
df.sample(20)

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id,capital_marker,timezone,geo_lat,geo_lon,population,timestamp,user_id_mean_travel_distance
21365767,Амурская область,Благовещенск,Samsung,Galaxy J3 LTE Dual,avatars.mds.yandex.net,smartphone,Android,8990.0,2021-06-25,day,1,362500,1.0,5,55.049988,55.955318,34246,162457920.0,0.000000
30215333,Санкт-Петербург,Санкт-Петербург,Samsung,Galaxy S20 FE,ad.mail.ru,smartphone,Android,39884.0,2021-07-27,day,1,399114,0.0,3,59.939133,30.315901,4848742,162734400.0,0.000000
2918554,Пермский край,Пермь,Apple,iPhone SE,ads.adfox.ru,smartphone,iOS,11238.0,2021-06-28,evening,1,65508,2.0,5,58.010258,56.234203,1000679,162483840.0,490.501892
12957550,Москва,Москва,Xiaomi,Redmi 9A Global,ok.ru,smartphone,Android,7992.0,2022-03-02,day,1,102868,0.0,3,55.754047,37.620403,11514330,164617920.0,143.053009
4421630,Липецкая область,Елец,Apple,iPhone 11,avatars.mds.yandex.net,smartphone,iOS,57748.0,2021-06-22,evening,1,297288,0.0,3,52.615242,38.528934,108404,162432000.0,111.135948
7930883,Республика Татарстан,Нижнекамск,Xiaomi,Redmi 9C NFC,s0.2mdn.net,smartphone,Android,9481.0,2021-08-17,day,1,283394,1.0,3,55.631359,51.814468,234108,162915840.0,0.000000
10339639,Хабаровский край,Хабаровск,Huawei,Honor 9A,ad.mail.ru,smartphone,Android,10971.0,2021-07-13,morning,3,159436,2.0,10,48.464725,135.059891,577668,162613440.0,0.000000
2729037,Московская область,Жуковский,Apple,iPhone XR,instagram.com,smartphone,iOS,45040.0,2021-07-04,night,1,58621,0.0,3,55.599804,38.122429,102729,162535680.0,22.988951
17193330,Амурская область,Благовещенск,Huawei,Honor 9 Dual,yandex.ru,smartphone,Android,21275.0,2021-07-14,day,3,331163,1.0,5,55.049988,55.955318,34246,162622080.0,4788.362793
31888723,Москва,Москва,Huawei,Mate 20,smi2.ru,smartphone,Android,9990.0,2021-08-05,evening,1,154435,0.0,3,55.754047,37.620403,11514330,162812160.0,34.641422


In [ ]:
lat_lon = np.stack(df['lat_lon'].to_numpy())

In [ ]:
haversine_np(lat_lon[:])

In [ ]:
df = df.merge(, how='left', on='user_id')

In [ ]:
distances

In [ ]:
def get_agg_aot(df: pd.DataFrame,
                agg_col: str = "user_id",
                target_col: str = 'city_name',
                timestamp_col: str = 'timestamp',
                alias: str = None,
                sort: bool = False) -> pd.DataFrame:
    if alias:
        col_name = alias
    else:
        col_name = f'{agg_col}_amount_of_travel'

    df = df.merge(df.sort_values(timestamp_col).groupby(agg_col)[target_col].agg(
    amount_of_travel  = get_travel
    ).rename(columns={'amount_of_travel':col_name})
             , how='left', on=agg_col)
    if sort:
        return df.sort_values(by=agg_col)

    return df

In [ ]:
get_agg_aot(df)